<a href="https://colab.research.google.com/github/sgshrigouri/myportfolio/blob/main/LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 843.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [16]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Install required packages
!pip install transformers torch sacremoses

try:
    # Load BioGPT model
    model_name = "microsoft/BioGPT"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
except Exception as e:
    print(f"Error loading BioGPT: {e}. Falling back to gpt2.")
    model_name = "gpt2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

# Set pad_token to eos_token if not defined
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Define device (use GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to generate chatbot responses
def medical_chatbot(user_input, context="You are a friendly medical assistant for non-medical users. Provide accurate, concise, and easy-to-understand answers about medical symptoms, causes, or treatments. Always use bullet points for lists (e.g., '- Pain'). Use simple, everyday language and avoid technical terms like 'viral,' 'gastric,' or 'biomedical.' Do not speculate, mention studies, tables, or research. If unsure or the input is vague (e.g., 'what'), ask for clarification or suggest consulting a doctor. Examples: For 'symptoms of stomach pain,' answer with '- Cramping\n- Nausea.' For 'what to do for headache,' answer with '- Drink water\n- Rest.' Always end with a disclaimer to consult a doctor."):
    # Clean user input
    user_input = user_input.strip().lower()
    if not user_input:
        return "Please provide a valid input."
    if user_input in ['what', 'huh', 'what?']:
        return "Please clarify your question (e.g., 'What are symptoms of fever?'). Always consult a doctor for professional medical advice."

    # Create prompt with clear structure
    prompt = f"{context}\nUser: {user_input}\nAI: "

    try:
        # Tokenize input
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        # Generate model output
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=100,  # Balanced for complete responses
            temperature=0.5,  # Moderate for coherence
            top_p=0.9,  # Balanced for diversity
            repetition_penalty=1.5,  # Moderate to prevent repetition
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

        # Decode and clean response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        response = response.split("AI:")[-1].strip()

        # Debug: Print raw response (optional)
        # print(f"Raw response: {response}")

        # Remove trailing incomplete sentences
        if response and not response.endswith('.'):
            last_period = response.rfind('.')
            if last_period != -1:
                response = response[:last_period + 1]

        # Format response with bullet points for lists
        if any(keyword in response.lower() for keyword in ['include', 'such as', 'may be', 'can be', 'are']) or 'symptoms' in user_input or 'what to do' in user_input or 'cure' in user_input:
            lines = response.split('. ')
            formatted_response = []
            bullet_mode = False
            for line in lines:
                line = line.strip()
                if not line:
                    continue
                if any(keyword in line.lower() for keyword in ['include', 'such as', 'may be', 'can be', 'are']) or bullet_mode:
                    bullet_mode = True
                    parts = line.split(':') if ':' in line else line.split(',')
                    if not formatted_response or formatted_response[-1].startswith('-'):
                        formatted_response.append("Symptoms can include:" if 'symptoms' in user_input else "Actions to help:" if 'what to do' in user_input or 'cure' in user_input else "Possible causes:")
                    for item in parts:
                        item = item.strip()
                        if item:
                            formatted_response.append(f"- {item}")
                else:
                    bullet_mode = False
                    if line:
                        formatted_response.append(line)
            response = '. '.join(formatted_response)

        # Handle empty or incoherent responses
        if not response or len(response.split()) < 5 or any(term in response.lower() for term in ['study', 'table', 'abstract', 'viral', 'gastric', 'biomedical', 'ai ', 'algorithm', 'app ', 'syndrome', 'reference', 'http']):
            return f"I don't have a clear answer for '{user_input}'. Please rephrase or ask something else. Always consult a doctor for professional medical advice."

        return f"{response}. Always consult a doctor for professional medical advice."
    except Exception as e:
        return f"Error generating response: {e}. Always consult a doctor for professional medical advice."

# Chat loop with improved exit handling
print(f"Medical Chatbot using {model_name} (type 'exit', 'quit', or 'stop' to end)")
while True:
    user_input = input("You: ")
    user_input_lower = user_input.strip().lower()
    if user_input_lower in ['exit', 'quit', 'stop']:
        print("Goodbye!")
        break
    reply = medical_chatbot(user_input)
    print("Bot:", reply)

config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/927k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/696k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

Medical Chatbot using microsoft/BioGPT (type 'exit', 'quit', or 'stop' to end)
You: what are symptoms of fever
Bot: Symptoms can include:. - flu? "How can you diagnose acute bacterial sinusitis; how many patients have asthma / rhinitis but don't know if they suffer from it? What antibiotics should be prescribed in case there's an infection at one site such as pharyngitis • Can your patient request treatment by using drugs that may cause side effects on other people during pregnancy when their parents want them another drug than penicillin G? When does my child need antibiotic therapy after having been given amoxicillin 3 times daily because she has developed pneumonia due to Streptococcus pneumoniae serotype. Always consult a doctor for professional medical advice.
You: what to do for stomach pain
Bot: Actions to help:. - To treat it? A brief description of some common conditions that can be treated by acupuncture; how I prescribe an effective treatment option • The article also provid